In [1]:
import pickle
import numpy as np
import cv2
graph_datasets=pickle.load(open("bcc/graph_datasets/imagenet_graph_data_256_new.pkl",'rb'))

In [9]:
n=200
patch_size=256
imgs=[]
labels=[]
for ID,dff in graph_datasets['df'].groupby("ID"):
    arr=np.load(f"bcc/inputs/{ID}.npy")
    dfff=dff.groupby("annotation").apply(lambda x: x.sample(n=min(n,x.shape[0]))).sample(frac=1.)
    labels.extend(dfff['annotation'].values.tolist())
    imgs.append([arr[x:x+patch_size,y:y+patch_size] for x,y in dfff[['x','y']].values.tolist()])
    del arr
#         for lbl,dfff in 
#         labels.append()
#     del arr

In [13]:
imgs2=np.concatenate(imgs)

In [14]:
imgs2.shape

(17778, 256, 256, 3)

In [15]:
from sklearn.preprocessing import LabelEncoder
y=LabelEncoder().fit_transform(labels)

In [16]:
y

array([0, 0, 1, ..., 0, 1, 0])

In [17]:
X=imgs2

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,random_state=42,stratify=y,train_size=0.7)

In [23]:
import torch
from torch.utils.data import TensorDataset
train_data=TensorDataset(torch.tensor(X_train).permute((0,3,1,2))/255.,torch.tensor(y_train).long())
val_data=TensorDataset(torch.tensor(X_val).permute((0,3,1,2))/255.,torch.tensor(y_val).long())

In [28]:
# ! free -g#mkdir bcc/tensor_data
import os
os.makedirs("bcc/tensor_data")

In [29]:
torch.save(train_data,"bcc/tensor_data/train_data.pth")
torch.save(val_data,"bcc/tensor_data/val_data.pth")

In [31]:
len(torch.load("bcc/tensor_data/train_data.pth"))

12444

OSError: [Errno 12] Cannot allocate memory